# ip_mac_port 對照表


### 功能概述
先在PC上架kiwi syslog server，把自己電腦ip設為L2 switch的syslog 接收端。<br>
接著在L3 switch中抓ip-mac，即可知道各個port的對應ip。<br>
用來確認lab目前用了哪些ip，以及是誰在使用。<br>


### 事前準備
本程式使用.env載入目標ip、port與帳號密碼，請在同個目錄下建立`.env`檔案，並填上以下值:
L3_IP='xxx.xxx.xxx.xxx'<br>
L3_USER_NAME='xxx'<br>
L3_USER_PWD='xxx'<br>
L3_ENABLE_PWD='xxx'

並且，

### TODO
COM 設定要放前面 

## step1. 連進L3 switch，並取得ip-mac table


In [ ]:
from dotenv import load_dotenv
from netmiko import ConnectHandler
import pandas as pd
from datetime import datetime
import os
import re

load_dotenv(verbose=True, override=True)

dev = {
    "device_type": "ruckus_fastiron",
    "host": os.getenv('L3_IP'),
    "username": os.getenv('L3_USER_NAME'),
    "use_keys": False,
    "password": os.getenv('L3_USER_PWD'),      
    "fast_cli": True,     # 加快互動
    "global_delay_factor": 1.0,
    "secret":os.getenv('L3_ENABLE_PWD'),
    "session_log": "netmiko_session.log",
}
exp_str = os.getenv('L3_SWITCH_NAME')+"#"

with ConnectHandler(**dev) as conn: # dev裡的東西當參數call ConnectHandler，開啟連線
    conn.enable()
    conn.send_command("skip-page-display") # skip-page-display:ruckus指令，關閉CLI分頁，下次輸入指令會一次吐完，不會有 --More--（此指令需enable。

    # 跑顯示時間，確定可以顯示
    output = conn.send_command("show clock", expect_string=exp_str)
    print(output)
    # 讀取arp
    output = conn.send_command("show arp", expect_string=exp_str)
    print(output)
output = output.splitlines()
# 有些空一格是分隔，有些卻是words，太難拆只好這麼做
header_line = re.split(r"\s{2,}", "No.   IP Address       MAC Address    Type     Age   Port           Status".strip()) 
data_lines = output[4:]
rows=[]
for line in data_lines:
    parts = line.split()
    if len(parts) >= len(header_line):
        rows.append(dict(zip(header_line, parts))) # zip tuple化(給properties name)，再轉成json object形式


df = pd.DataFrame(rows)

filename = f"arp_{datetime.now():%Y%m%d}.csv"
df.to_csv(filename, index=False, encoding="utf-8")
print(f"儲存arp:{filename}")


15:37:10.552 GMT+08 Mon Sep 15 2025

Total number of ARP entries: 432
Entries in default routing instance:
No.   IP Address       MAC Address    Type     Age Port               Status
1     140.123.9.19     cc4e.24dd.6e6a Dynamic  1    lg1               Valid 
2     140.123.9.21     6c41.6a25.3b41 Dynamic  2    lg1               Valid 
3     140.123.9.24     609c.9f43.5704 Dynamic  0    lg1               Valid 
4     140.123.9.36     78a6.e12a.7188 Dynamic  2    lg1               Valid 
5     140.123.9.39     cc4e.24dd.6f42 Dynamic  0    lg1               Valid 
6     140.123.9.40     cc4e.24dd.8ec2 Dynamic  0    lg1               Valid 
7     140.123.9.47     00a2.8978.61f0 Dynamic  2    lg1               Valid 
8     140.123.9.78     cc4e.24dd.8676 Dynamic  2    lg1               Valid 
9     140.123.9.95     8c60.4faf.083c Dynamic  0    lg1               Valid 
10    140.123.9.174    d4c1.9e6c.60f4 Dynamic  1    lg1               Valid 
11    140.123.9.206    cc4e.24dc.fa42 Dynamic

## step2. 連進在lab的L2 switch，並取得ip-mac table
在執行這些程式前，請先透過serial 連進switch，進行login、enable、skip-page-display等指令。

In [2]:
import re
import time
import serial
from datetime import datetime
from pathlib import Path

PORT = "COM6"              # Linux: "/dev/ttyUSB0"  macOS: "/dev/tty.usbserial-xxxxx"
BAUD = 9600                # ICX 預設 9600 8N1
OUTPUT_DIR = "."           # 輸出目錄

PROMPT_RE = re.compile(rb"[>#]\s?$")  # 視你的提示符調整
READ_TIMEOUT = 20                     # 大量輸出時可再調大

def open_serial():
    return serial.Serial(
        PORT, baudrate=BAUD, bytesize=8, parity="N", stopbits=1,
        timeout=0, write_timeout=2
    )

def sendline(ser, s):
    ser.write((s + "\r\n").encode("utf-8"))

def read_until_prompt(ser, timeout=READ_TIMEOUT):
    buf = b""
    deadline = time.time() + timeout
    more_re = re.compile(rb"--More--|<space>|<SPACE>", re.I)
    ansi = re.compile(rb"\x1B\[[0-?]*[ -/]*[@-~]")

    while time.time() < deadline:
        n = ser.in_waiting or 1
        chunk = ser.read(n)
        if chunk:
            buf += chunk
            cleaned = ansi.sub(b"", buf)
            if more_re.search(cleaned):
                ser.write(b" ")  # 自動吃分頁
                continue
            if PROMPT_RE.search(cleaned):
                return cleaned
        else:
            time.sleep(0.02)
    raise TimeoutError("讀取逾時，可能沒有看到提示符。")

def wake_prompt(ser):
    # 喚醒：送幾次 CR；有些機器要按一下才會回 prompt
    for _ in range(3):
        sendline(ser, "")
        try:
            out = read_until_prompt(ser, timeout=3)
            if PROMPT_RE.search(out):
                return
        except TimeoutError:
            pass
    # 最後再試一次，若失敗就讓上層丟錯
    read_until_prompt(ser, timeout=5)

def main():
    ser = open_serial()
    try:
        wake_prompt(ser)

        # 關閉分頁（有的裝置沒有此指令，失敗忽略）
        for cmd in ("skip-page-display", "terminal length 0"):
            sendline(ser, cmd)
            try:
                read_until_prompt(ser, timeout=3)
            except TimeoutError:
                pass

        # 送出目標指令
        sendline(ser, "show mac-address")
        output = read_until_prompt(ser, timeout=READ_TIMEOUT)

        # 存檔（原始輸出）
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = Path(OUTPUT_DIR) / f"mac_table_{ts}.txt"
        out_path.write_bytes(output)

        print(f"Saved to: {out_path.resolve()}")

    finally:
        ser.close()

if __name__ == "__main__":
    main()

Saved to: C:\Users\user\Desktop\python\L2L3Switch\mac_table_20250915_153721.txt


## step3. 處理step2抓到的資料
擷取`Total active entries from all ports`~`ICX7150-48 Switch#`之間的東西，並忽略`MAC-Address     Port                 Type         VLAN `



In [2]:
import glob
import os
from datetime import datetime
import re
import pandas as pd
files = glob.glob("mac_table_*")
latest_file = max(files, key=os.path.getmtime)
mac_port = []
header_line = re.split(r"\s{2,}", "MAC-Address     Port                 Type         VLAN".strip()) 
with open(latest_file, "r") as file:
    #忽略所有訊息，直到出現Total active entries from all ports
    line = file.readline()
    while("Total active entries from all ports" not in line):
        line = file.readline()



    
    while(True):
        line = file.readline()
        if("MAC-Address     Port " in line):
            continue
        if("ICX7150-48 Switch" in line):
            break
        parts = line.split()
        mac_port.append(dict(zip(header_line, parts)))

df = pd.DataFrame(mac_port)

filename = f"mac-port_{datetime.now():%Y%m%d}.csv"
df.to_csv(filename, index=False, encoding="utf-8")
print(f"儲存mac-port:{filename}")

儲存mac-port:mac-port_20251005.csv


## step4. 維護MAC table
讀取mac-port.csv，如果沒有就建立。<br>
mac-port.csv負責維護目前為止的mac 和 ip資訊，記錄各個裝置最後出現在哪個port 上，以及使用哪個ip。<br>
會取最新的mac-port_date來更新內容，最終生成mac-port-ip table。



In [5]:
import os
import pandas as pd

files = glob.glob("mac-port*")
latest_file = max(files, key=os.path.getmtime)

# 開啟最新mac-port並載入資料進df
mac_port_df = pd.read_csv(latest_file, encoding="utf-8")

# 開啟最新mac-ip並載入資料進df

files = glob.glob("arp_*")
latest_file = max(files, key=os.path.getmtime)
mac_ip_df = pd.read_csv(latest_file, encoding="utf-8")

# 讀取之前的mac-port-ip
filename = "mac-port-ip.csv"
mac_port_ip_df = pd.DataFrame(columns=['MAC-Address', 'Port_x', 'Type_x', 'VLAN', 'No.', 'IP Address',
       'MAC Address', 'Type_y', 'Age', 'Port_y', 'Status'])
if os.path.exists(filename):
    mac_port_ip_df = pd.read_csv(filename, encoding="utf-8")


# 整理最新mac-port資訊，left join mac-ip中的資訊
df_joined = pd.merge(mac_port_df, mac_ip_df, left_on="MAC-Address", right_on="MAC Address", how="left")
mac_port_ip_df = mac_port_ip_df.set_index("MAC-Address")
df_joined = df_joined.set_index("MAC-Address")

# 更新 (只有交集會被更新)
mac_port_ip_df.update(df_joined)

# 找出 df_joined 中新的 (不在 mac_port_ip_df 裡)
new_rows = df_joined.loc[~df_joined.index.isin(mac_port_ip_df.index)]

# 合併
mac_port_ip_df = pd.concat([mac_port_ip_df, new_rows])

# reset 回來
mac_port_ip_df = mac_port_ip_df.reset_index()

mac_port_ip_df.to_csv(filename, index=False, encoding="utf-8")
print(f"儲存mac-port:{filename}")



儲存mac-port:mac-port-ip.csv
